In [1]:
# default_exp tearsheets

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# export

import pyfolio
from pyfolio.timeseries import perf_stats, gen_drawdown_table
import pandas as pd
from path import Path
from pprint import pprint
from functools import reduce
import datetime
import logging
import re

import simplejson as json

from mlbt.historical_bt import simulate_pnl
from mlbt.pnl_sim import get_pnl_reports

try:
    from mlbt import settings_pers as settings
except:
    from mlbt import settings

FORMAT = "%(asctime)-15s %(message)s"
logging.basicConfig(format=FORMAT, level=logging.INFO)

def mean_of_dfs(dfs):
    return reduce(lambda left, right: left.add(right), dfs) / len(dfs)

def make_default_config(**data):
    DATA_DIR = data.get("DATA_DIR", settings.DATA_DIR)

    return {
        "DATA_DIR": DATA_DIR,
        "F_PAYLOAD_DIR": data.get("F_PAYLOAD_DIR", settings.F_PAYLOAD_DIR),
        "symbols_map": pd.read_csv(DATA_DIR / "symbols.csv", index_col="iqsymbol")
    }


def create_frontend_payload_multi(file_names, force=False, our_config=None):
    if our_config is None:
        our_config = make_default_config()

    new_file_name = our_config["F_PAYLOAD_DIR"] / "f_payload_MULTI.json"

    
def abort_early(force, file_name):
    have_file = False
    if not force:
        try:
            # Check if we already have the file we want to create
            with open(file_name) as f:
                have_file = bool(json.load(f))
        except:
            pass
        if have_file:
            return True
    return False
    


def load_payloads(file_names, our_config):
    res = []
    for file_name in file_names:
        with open(file_name) as f:
            pay_j = json.load(f)

        events = pd.DataFrame.from_dict(pay_j["events"])
        events = events.set_index(pd.to_datetime(events.index))
        events["t1"] = pd.to_datetime(events["t1"])
        config = get_config(pay_j, file_name)
        config = {**config, **our_config}

        closes, clf_signals, alpha_signals = get_pnl_reports(
            events,
            pay_j["symbols"],
            config["binarize"],
            config["binarize_params"],
        )
        logging.info(f"clf_signals shape {clf_signals.shape}")

        primary_signals, secondary_signals = (
            (alpha_signals, clf_signals)
            if alpha_signals is not None
            else (clf_signals, alpha_signals)
        )
        res.append((config, pay_j, closes, primary_signals, secondary_signals))

    return res

def join_signals(dfs):
    cols = dfs[0].columns
    signals = []
    for col in cols:
        symbol_signals = pd.concat([df[col] for df in dfs], axis=1).ffill().mean(axis=1)
        signals.append(symbol_signals)
    res = pd.concat(signals, axis=1).ffill()
    res.columns = cols
    return res


def create_frontend_payload(file_names, force=False, our_config=None):
    combine_multi = len(file_names) > 1
    if our_config is None:
        our_config = make_default_config()
    
    if combine_multi:
        new_file_name = our_config["F_PAYLOAD_DIR"] / f"f_payload_MULTI_{len(file_names)}.json"
    else:
        new_file_name = our_config["F_PAYLOAD_DIR"] / file_name.basename().replace("payload_", "f_payload_", 1)
    
    if abort_early(force, new_file_name):
        return
    
    payloads = load_payloads(file_names, our_config)
    configs, pay_js, closes, primary_signals, secondary_signals = zip(*payloads)
    config = configs[0]
    pay_j = pay_js[0]
    closes = closes[0]

    primary_signals = join_signals(primary_signals)
    if secondary_signals[0]:
        secondary_signals = join_signals(secondary_signals)
    
    primary_rets, pay_j["primary"]["pnl"] = create_tearsheet(
        config, closes, primary_signals, new_file_name, "primary"
    )
    if pay_j["secondary"]:
        _, pay_j["secondary"]["pnl"] = create_tearsheet(
            config, closes, secondary_signals, new_file_name, "secondary", primary_rets
        )

    # Delete stuff we don't want in the frontend payload
    del pay_j["events"]
    del pay_j["symbols"]

    logging.info(f"Writing f_payload at {new_file_name}")
    with open(new_file_name, "w") as f:
        json.dump(pay_j, f, ignore_nan=True, default=datetime.datetime.isoformat)
    return new_file_name

    

def calc_returns(df):
    df = df.resample("1B").last()

    if str(df.index.tz) != "UTC":
        df.index = df.index.tz_localize(tz="UTC")

    return df.pct_change()


def create_tearsheet(config, close, signal, file_name, report_type, benchmark_rets=None):
    logging.info(f"Creating {report_type} tearsheet for {file_name}")
    # Map long/short to long/flat
    signal = (signal + 1) / 2
    pos_size = 20000
    df_net, df_gross, cost_stats = simulate_pnl(config, close, signal, pos_size)
    returns_net = calc_returns(df_net)
    returns_net.name = report_type.title()
    returns_gross = calc_returns(df_gross)
    returns_gross.name = report_type.title()

    if report_type == "primary":
        long_all = pd.DataFrame(1, columns=signal.columns, index=signal.index)
        df_bench, _, _ = simulate_pnl(config, close, long_all, pos_size)
        benchmark_rets = calc_returns(df_bench)
        benchmark_rets.name = "Benchmark (long all)"

    
    fig = pyfolio.create_returns_tear_sheet(
        returns_net, benchmark_rets=benchmark_rets, return_fig=True
    )
    fig_file_name = file_name.replace(".json", f"_{report_type}.png")
    fig.savefig(fig_file_name, bbox_inches="tight", pad_inches=0)

    p_stats = perf_stats(returns_net)
    p_stats_wo_costs = perf_stats(returns_gross)
    dd_table = gen_drawdown_table(returns_net, 5)

    signal = signal.resample("1B").last()
    # Just-in-case normalize to 1 for reporting
    signal = signal / signal.max().max()
    signal.plot()
    signal = signal.set_index(signal.index.map(lambda x: x.isoformat()))
    

    return (
        returns_net,
        {
            "fig_file_name": str(Path(fig_file_name).basename()),
            "p_stats": p_stats.to_dict(),
            "p_stats_wo_costs": p_stats_wo_costs.to_dict(),
            "dd_table": dd_table.to_dict(),
            "signal": signal.to_csv(),  # CSVs are a lot more space-efficient for this dense 1500*50 table
            "cost_stats": cost_stats,
        },
    )


def get_config(payload, fn):
    if "config" in payload:
        return payload["config"]

    # bridge to the old payload format
    if "fixed_horizon" in fn:
        return {
            "binarize": "fixed_horizon",
            "binarize_params": int(re.findall(r"fixed_horizon_(\d+)", fn)[0]),
        }
    return {}

C:\Users\doda\Anaconda3\envs\fincl2\lib\site-packages\pyfolio\pos.py:28: UserWarning: Module "zipline.assets" not found; mutltipliers will not be applied to position notionals.
  ' to position notionals.'
